## 1. Installation

In [1]:
!pip install PyPortfolioOpt
!git clone https://github.com/robertmartin8/PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 9.0 MB/s eta 0:00:00
Cloning into 'PyPortfolioOpt'...
remote: Enumerating objects: 4385, done.
remote: Counting objects: 100% (1107/1107), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 4385 (delta 951), reused 861 (delta 861), pack-reused 3278 (from 2)
Receiving objects: 100% (4385/4385), 12.20 MiB | 12.51 MiB/s, done.
Resolving deltas: 100% (3080/3080), done.


## 2. Processing historical prices

2 main types of data required to perform an optimization
- `data` is the multi assets prices required to perform M.V.O
- `market_prices` is the benchmark market data eg. S&P index

In [3]:
import pandas as pd

data = pd.read_csv('PyPortfolioOpt/tests/resources/stock_prices.csv', parse_dates=True, index_col="date")
# Sampling data
# data = data.iloc[:, :5]
# data = data.tail(738)
market_prices = pd.read_csv('PyPortfolioOpt/tests/resources/spy_prices.csv', parse_dates=True, index_col="date")

close_prices = pd.DataFrame({
    asset: data["close"]
    for asset, df in data.items()
})


data.tail(10)

## 3. Optimizers

### 3.1 M.V.O Optimizer

In [4]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions, black_litterman, risk_models
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.black_litterman import BlackLittermanModel
from pprint import pprint

def mvo_optimizer(dfs):
  # dfs = multi assets data 

  mu = mean_historical_return(dfs)
  S = CovarianceShrinkage(dfs).ledoit_wolf()

  ef = EfficientFrontier(mu, S)

  w = ef.max_sharpe()
  print('Initial Raw weights \n')
  pprint(w)

  # Dealing with many negligible weights
  # ef.add_objective(objective_functions.L2_reg, gamma=0.1)
  # cleaned_weights = ef.clean_weights()
  #ef.save_weights_to_file("weights.txt")  # saves to file
  # print('fewer negligible weights than before \n')
  # pprint(cleaned_weights)

  # Shorting
  #ef.efficient_return(target_return=0.2, market_neutral=True)

  # convert these weights into an actual allocation
  latest_prices = get_latest_prices(dfs)
  da = DiscreteAllocation(w, latest_prices, total_portfolio_value=20000)
  allocation, leftover = da.lp_portfolio()
  print(allocation)

  return allocation

mvo_optimizer(dfs=data)

Initial Raw weights 

OrderedDict([('GOOG', 0.0237526729397794),
             ('AAPL', 0.0125639275319764),
             ('FB', 0.2778077660738064),
             ('BABA', 0.412171688987524),
             ('AMZN', 0.0112713791135148),
             ('GE', 0.0),
             ('AMD', 0.0),
             ('WMT', 0.0),
             ('BAC', 0.0),
             ('GM', 0.0),
             ('T', 0.0),
             ('UAA', 0.0),
             ('SHLD', 0.0),
             ('XOM', 0.0),
             ('RRC', 0.0),
             ('BBY', 0.0019599369127201),
             ('MA', 0.2144083174755166),
             ('PFE', 0.0445401206358982),
             ('JPM', 0.0),
             ('SBUX', 0.0015241903292643)])
{'AAPL': 2, 'FB': 34, 'BABA': 47, 'GE': 2, 'AMD': 2, 'GM': 1, 'T': 2, 'UAA': 4, 'SHLD': 4, 'RRC': 3, 'BBY': 1, 'MA': 25, 'PFE': 25, 'SBUX': 1}


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'AAPL': 2,
 'FB': 34,
 'BABA': 47,
 'GE': 2,
 'AMD': 2,
 'GM': 1,
 'T': 2,
 'UAA': 4,
 'SHLD': 4,
 'RRC': 3,
 'BBY': 1,
 'MA': 25,
 'PFE': 25,
 'SBUX': 1}

In [13]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

def blm_optimizer(dfs, mcaps, viewdict):
  # dfs = multi assets data 
  cov_matrix = CovarianceShrinkage(dfs).ledoit_wolf()
  bl = BlackLittermanModel(cov_matrix, absolute_views=viewdict)

  rets = bl.bl_returns()
  ef = EfficientFrontier(rets, cov_matrix)

  # OR use return-implied weights
  delta = black_litterman.market_implied_risk_aversion(mcaps)
  bl.bl_weights(delta)
  w = bl.clean_weights()
  pprint(w)

  latest_prices = get_latest_prices(dfs)
  da = DiscreteAllocation(w, latest_prices, total_portfolio_value=20000)
  allocation, leftover = da.lp_portfolio()
  print(allocation)

viewdict = {"AAPL": 0.20, "BBY": -0.30, "BAC": 0, "SBUX": -0.2, "T": 0.15}
blm_optimizer(dfs=data, mcaps=market_prices, viewdict)

OrderedDict([('GOOG', 0.0),
             ('AAPL', 1.25547),
             ('FB', 0.0),
             ('BABA', 0.0),
             ('AMZN', 0.0),
             ('GE', 0.0),
             ('AMD', 0.0),
             ('WMT', 0.0),
             ('BAC', -0.00212),
             ('GM', 0.0),
             ('T', 2.83076),
             ('UAA', 0.0),
             ('SHLD', 0.0),
             ('XOM', 0.0),
             ('RRC', 0.0),
             ('BBY', -1.31329),
             ('MA', 0.0),
             ('PFE', 0.0),
             ('JPM', 0.0),
             ('SBUX', -1.77083)])
{'AAPL': 36, 'T': 391, 'BAC': -2, 'BBY': -370, 'SBUX': -596}


/usr/local/lib/python3.10/dist-packages/pypfopt/black_litterman.py:258: UserWarning: Running Black-Litterman with no prior.
  warnings.warn("Running Black-Litterman with no prior.")
